In [5]:
import pymysql

# Install MySQLdb adapter for compatibility with pymysql
pymysql.install_as_MySQLdb()

def fetch_redbus_data():
    """
    Connects to the MySQL database, fetches all records from the 'redbus1' table,
    and prints the retrieved data.

    This function assumes that the MySQL server is running locally with default credentials.
    """
    try:
        # Establish a connection to the MySQL database
        conn = pymysql.connect(host='localhost', database='guvi', user='root', password='root')
        cursor = conn.cursor()

        # Execute a query to fetch all records from the 'redbus1' table
        cursor.execute("SELECT * FROM redbus")

        # Fetch all rows from the result of the query
        rows = cursor.fetchall()

        # Print the fetched rows
        print(rows)

    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")

    finally:
        # Ensure that the connection is closed after the operation
        if conn:
            conn.close()


# Run the function to fetch data
if __name__ == "__main__":
    fetch_redbus_data()


((372, None, 'https://redbus.in/bus-tickets/delhi-to-lucknow', 'Gola Bus Service', 'VE AC Seater / Sleeper (2+1)', datetime.timedelta(seconds=81000), '08h 00m', datetime.timedelta(seconds=23400), 0.0, Decimal('499.00'), 25, None), (373, None, 'https://redbus.in/bus-tickets/delhi-to-lucknow', 'PTC-SKYBUS', 'VE AC Seater / Sleeper (2+1)', datetime.timedelta(seconds=85800), '07h 40m', datetime.timedelta(seconds=27000), 0.0, Decimal('700.00'), 30, None), (374, None, 'https://redbus.in/bus-tickets/delhi-to-lucknow', 'IntrCity SmartBus', 'Bharat Benz AC Seater /Sleeper (2+1)', datetime.timedelta(seconds=76800), '09h 15m', datetime.timedelta(seconds=23700), 0.0, Decimal('579.00'), 30, None), (375, None, 'https://redbus.in/bus-tickets/delhi-to-lucknow', '', '', datetime.timedelta(0), '', datetime.timedelta(0), 0.0, Decimal('0.00'), 0, None), (376, None, 'https://redbus.in/bus-tickets/delhi-to-lucknow', '', '', datetime.timedelta(0), '', datetime.timedelta(0), 0.0, Decimal('0.00'), 0, None), (3

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, WebDriverException
import time
import re

# Set up WebDriver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")

# Wait for RTC name elements to be present
rtc_name_elements = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "rtcName"))
)

# Loop through all RTC names
for rtc_index in range(len(rtc_name_elements)):
    rtc_element = rtc_name_elements[rtc_index]
    
    # Scroll to RTC name element and wait for it to be clickable
    driver.execute_script("arguments[0].scrollIntoView(true);", rtc_element)
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, "rtcName")))

    # Attempt to click the RTC name, with retries in case of interception
    attempt = 0
    max_attempts = 3
    while attempt < max_attempts:
        try:
            rtcName=rtc_element.text
            print(f"\nClicking RTC name {rtc_index + 1}: {rtc_element.text}")
            driver.execute_script("arguments[0].click();", rtc_element)  # JavaScript click as a workaround
            break  # Exit loop if click succeeds
        except ElementClickInterceptedException:
            attempt += 1
            print(f"Click intercepted. Retrying... (Attempt {attempt} of {max_attempts})")
            time.sleep(1)  # Short delay before retrying
    else:
        print(f"Failed to click RTC name {rtc_index + 1} after {max_attempts} attempts.")
        continue  # Move to the next RTC name if click was unsuccessful

    # Wait for the route elements on the second page to load
    WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "route"))
    )

    # Pagination and Route Details Extraction
    while True:
        # Print route details for the current page
        route_elements = driver.find_elements(By.CLASS_NAME, "route")
        print(f"Routes found for RTC name {rtc_index + 1}:")
        for route in route_elements:
            print("Bus Route Name:", route.text)
            route_name=route.text
            ticket_link = f"https://redbus.in/bus-tickets/{route.text.replace(' ', '-').lower()}"
            print(f"Bus Route Link: {ticket_link}")
            
            # Open the ticket link in a new window
            driver.execute_script("window.open(arguments[0]);", ticket_link)
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[-1])
            
            try:
                # Wait until bus details are loaded on the ticket page
                WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")))

                # Scrape bus details on the ticket page
                bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
                bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
                departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
                duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
                reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
                star_rating_elements = driver.find_elements(By.CSS_SELECTOR, ".icon.icon-ic-star.d-block")
                price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
                seats_available_elements = driver.find_elements(By.CSS_SELECTOR, ".seat-left.m-top-30")

                print(f"\nScrapping Bus Details on the route {ticket_link}:")
                
                # Print details for each bus
                for bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seats_available in zip(
                    bus_name_elements, bus_type_elements, departing_time_elements, duration_elements, reaching_time_elements, 
                    star_rating_elements, price_elements, seats_available_elements):
                    
                    # Price and seat availability extraction with regex
                    price_value = re.search(r'\d+', price.text)
                    if price_value:
                        price_value = float(price_value.group())
                    else:
                        price_value = 0.0
                    
                    seats_available_value = re.search(r'\d+', seats_available.text)
                    if seats_available_value:
                        seats_available_value = int(seats_available_value.group())
                    else:
                        seats_available_value = 0
                    
                    # Print extracted details
                    print(f"Bus Name: {bus_name.text}, Bus Type: {bus_type.text}, Departing Time: {departing_time.text}, "
                          f"Duration: {duration.text}, Reaching Time: {reaching_time.text}, "
                          f"Star Rating: {star_rating.text if star_rating else 'N/A'}, Price: {price_value}, "
                          f"Seats Available: {seats_available_value}")
                    bus_name_value = bus_name.text
                    bus_type_value = bus_type.text
                    departing_time_value = departing_time.text
                    duration_value = duration.text
                    reaching_time_value = reaching_time.text
                    star_rating_value = 0.0 #star_rating.text
                    
                   
                    route_link=ticket_link
                    

                   # Prepare the INSERT INTO statement
                    insert_query = """
                    INSERT INTO redbus (state,route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seats_available)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)

                    """

                    # Execute the query with the extracted data
                    cursor.execute(insert_query, (rtcName,route_name,route_link,bus_name_value, bus_type_value, departing_time_value, duration_value, reaching_time_value, star_rating_value, price_value, seats_available_value))
                    # Commit the transaction to save the data in the database
                    conn.commit()
                    print("Data inserted successfully!")


            except (TimeoutException, WebDriverException) as e:
                print(f"Failed to find bus details on {ticket_link}. Error: {e}")
            
            # Close the ticket page and return to main route page
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        # Pagination handling: Click the "Next" button if available
        try:
            pagination_table = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "DC_117_paginationTable"))
            )
            next_page_button = None
            for page in pagination_table.find_elements(By.CLASS_NAME, "DC_117_pageTabs"):
                if page.text.lower() == "next":
                    next_page_button = page
                    break

            # If a "Next" button is found, click it to load the next page of routes
            if next_page_button:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                driver.execute_script("arguments[0].click();", next_page_button)
                
                # Wait for the new page to load routes
                WebDriverWait(driver, 30).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "route"))
                )
                time.sleep(1)  # Short delay for pagination loading
            else:
                break  # Exit pagination loop if no "Next" button found

        except TimeoutException:
            print("Pagination not found or no more pages to navigate.")
            break

    # Return to the main RTC page and refresh elements
    driver.back()
    rtc_name_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "rtcName"))
    )

# Close the driver after the task is complete
driver.quit()



Clicking RTC name 1: APSRTC
Routes found for RTC name 1:
Bus Route Name: Hyderabad to Vijayawada
Bus Route Link: https://redbus.in/bus-tickets/hyderabad-to-vijayawada

Scrapping Bus Details on the route https://redbus.in/bus-tickets/hyderabad-to-vijayawada:
Bus Name: FRESHBUS, Bus Type: Electric A/C Seater (2+2), Departing Time: 05:50, Duration: 06h 35m, Reaching Time: 12:25, Star Rating: , Price: 349.0, Seats Available: 0
Data inserted successfully!
Bus Name: FRESHBUS, Bus Type: Electric A/C Seater (2+2), Departing Time: 10:25, Duration: 07h 30m, Reaching Time: 17:55, Star Rating: , Price: 349.0, Seats Available: 0
Data inserted successfully!
Bus Route Name: Vijayawada to Hyderabad
Bus Route Link: https://redbus.in/bus-tickets/vijayawada-to-hyderabad

Scrapping Bus Details on the route https://redbus.in/bus-tickets/vijayawada-to-hyderabad:
Bus Name: IntrCity SmartBus, Bus Type: A/C Seater / Sleeper (2+1), Departing Time: 23:25, Duration: 06h 50m, Reaching Time: 06:15, Star Rating: , 

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF6CC8938A5+3004357]
	(No symbol) [0x00007FF6CC529970]
	(No symbol) [0x00007FF6CC3D582A]
	(No symbol) [0x00007FF6CC3C0B26]
	(No symbol) [0x00007FF6CC3C1FEF]
	(No symbol) [0x00007FF6CC3D5E43]
	(No symbol) [0x00007FF6CC3AF54D]
	(No symbol) [0x00007FF6CC459E89]
	(No symbol) [0x00007FF6CC44BBA0]
	(No symbol) [0x00007FF6CC417628]
	(No symbol) [0x00007FF6CC418791]
	GetHandleVerifier [0x00007FF6CC8BA00D+3161901]
	GetHandleVerifier [0x00007FF6CC90E060+3506048]
	GetHandleVerifier [0x00007FF6CC90400D+3465005]
	GetHandleVerifier [0x00007FF6CC680EEB+830987]
	(No symbol) [0x00007FF6CC53467F]
	(No symbol) [0x00007FF6CC5309D4]
	(No symbol) [0x00007FF6CC530B6D]
	(No symbol) [0x00007FF6CC520149]
	BaseThreadInitThunk [0x00007FFD9AE17374+20]
	RtlUserThreadStart [0x00007FFD9B17CC